# - Data Warehouse - inserted


### Het samenvoegen van bestaande tabellen voor het uiteindelijke 'Datawarehouse'

##### Importeren van benodigde dependencies

In [ ]:
import pandas as pd # type: ignore
import pyodbc # type: ignore

import warnings
warnings.filterwarnings("ignore")

# Importeren van de create_connection en run_query functies uit de database_utils.py file
from utils.database_utils import create_connection, run_query

##### 1 - Opbouwen van dataframes voor elke database-tabel

In [ ]:
sales_territory_df = run_query("SELECT * FROM Sales.SalesTerritory", "AdventureWorks2019")
sales_history_df = run_query("SELECT * FROM Sales.SalesTerritoryHistory", "AdventureWorks2019")

#### 2 - Data transformatie

Na het opbouwen van de dataframes voor elke database-tabel, kunnen we beginnen met het transformeren van de data. Dit omvat het samenvoegen of aanpassen van tabellen, het toepassen van filters, het uitvoeren van berekeningen, enzovoort.

In [ ]:
create_table_query = """
CREATE TABLE SalesTerritoryData (
    BusinessEntityID INT,
    TerritoryID INT,
    Name VARCHAR(255),
    CountryRegionCode VARCHAR(255),
    GroupName VARCHAR(255),
    SalesYTD FLOAT,
    SalesLastYear FLOAT,
    CostYTD FLOAT,
    CostLastYear FLOAT,
    StartDate DATETIME,
    EndDate DATETIME,
)
"""

# Create the table in SQL Server
db_name = "AdventureWorks2019"
conn, cursor = create_connection(db_name)
cursor.execute(create_table_query)
conn.commit()
conn.close()

In [ ]:
merged_df = pd.merge(sales_territory_df, sales_history_df, on='TerritoryID')
merged_df

#### 3 - Data loading

Na het transformeren van de data, kunnen we de resulterende dataframe in de doeltabel van ons datawarehouse laden.

In [ ]:
conn, cursor = create_connection(db_name)

for index, row in merged_df.iterrows():
    start_date_str = f"'{row['StartDate'].strftime('%Y-%m-%d %H:%M:%S')}'" if pd.notnull(row['StartDate']) else 'NULL'
    end_date_str = f"'{row['EndDate'].strftime('%Y-%m-%d %H:%M:%S')}'" if pd.notnull(row['EndDate']) else 'NULL'
    
    query = f"""
        INSERT INTO SalesTerritoryData (
            BusinessEntityID,
            TerritoryID, 
            Name, 
            CountryRegionCode, 
            GroupName, 
            SalesYTD, 
            SalesLastYear, 
            CostYTD, 
            CostLastYear,
            StartDate,
            EndDate
        ) 
        VALUES (
            {row['BusinessEntityID']},
            {row['TerritoryID']}, 
            '{row['Name']}', 
            '{row['CountryRegionCode']}', 
            '{row['Group']}', 
            {row['SalesYTD']}, 
            {row['SalesLastYear']}, 
            {row['CostYTD']}, 
            {row['CostLastYear']}, 
            {start_date_str}, 
            {end_date_str} 
        )
    """
    cursor.execute(query)

conn.commit()
conn.close()

**Note:** Voeg indien nodig zoveel Markdown- of codeblokken toe als nodig is.

#### 4 -  Data Quality Checks

Voeg controles toe om de kwaliteit van de gegevens te waarborgen voordat ze worden geladen in het datawarehouse:

In [ ]:
# Controleren op ontbrekende waarden
missing_values = merged_df.isnull().sum()

# Controleren op duplicaten
duplicate_rows = merged_df.duplicated().sum()

# Weergave van resultaten
print("Aantal ontbrekende waarden:", missing_values)
print("Aantal duplicaten:", duplicate_rows)

**Note:** Dit is optioneel, het leek mij opzich best handig om te doen.